In [1]:
!pip install facenet-pytorch==2.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.3 MB/s eta 0:00:00


In [2]:
# Import required libraries
import os
import pandas as pd
import shutil
from pathlib import Path
from tqdm.notebook import tqdm
import cv2
import torch
from facenet_pytorch import MTCNN
import json
from kaggle_secrets import UserSecretsClient

In [3]:
# Setup Kaggle API credentials
print("Setting up Kaggle credentials...")
os.makedirs("/root/.kaggle", exist_ok=True)

user_secrets = UserSecretsClient()
kaggle_token = {
    "username": user_secrets.get_secret("kaggle-username"),
    "key": user_secrets.get_secret("kaggle-api-key")
}

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_token, f)
os.chmod("/root/.kaggle/kaggle.json", 0o600)

Setting up Kaggle credentials...


In [4]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [5]:
# Initialize MTCNN for face detection 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(
    image_size=224,
    margin=0,
    min_face_size=20,
    thresholds=[0.6, 0.7, 0.7],
    factor=0.709,
    device=device,
    keep_all=False
)

/opt/conda/lib/python3.10/site-packages/facenet_pytorch/models/mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)
/opt/conda

In [6]:
# Set up paths
input_dataset = Path('/kaggle/input/liveness-detection-zalo-2022')
dataset_root = Path('/kaggle/working/Zalo_AIC_dataset')

In [7]:
# Create output directories
for folder in ['live', 'spoof']:
    (dataset_root / folder).mkdir(parents=True, exist_ok=True)

In [8]:
# Read labels and move videos to appropriate folders
print("\nMoving videos to live/spoof folders...")
label_file = input_dataset / 'train/train/label.csv'
labels_df = pd.read_csv(label_file)

for _, row in tqdm(labels_df.iterrows(), total=len(labels_df)):
    src = input_dataset / 'train/train/videos' / row['fname']
    if row['liveness_score'] == 1:
        dst = dataset_root / 'live' / row['fname']
    else:
        dst = dataset_root / 'spoof' / row['fname'] 
    shutil.copy2(str(src), str(dst))


Moving videos to live/spoof folders...


  0%|          | 0/1168 [00:00<?, ?it/s]

In [9]:
# Function to extract frames from video
def extract_frames(video_path, save_dir, start_idx):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return start_idx
        
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    
    idx = start_idx
    for sec in range(int(duration)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, sec * fps)
        ret, frame = cap.read()
        if ret:
            save_path = save_dir / f"{idx:06d}.jpg"
            cv2.imwrite(str(save_path), frame)
            idx += 1
            
    cap.release()
    return idx

In [10]:
# Extract frames from all videos 
print("\nExtracting frames from videos...")
next_idx = 1
for folder in ['live', 'spoof']:
    folder_path = dataset_root / folder
    videos = list(folder_path.glob('*.mp4'))
    
    for video in tqdm(videos, desc=f"Processing {folder} videos"):
        next_idx = extract_frames(video, folder_path, next_idx)
        video.unlink()  # Delete video after extracting frames


Extracting frames from videos...


Processing live videos:   0%|          | 0/598 [00:00<?, ?it/s]

Processing spoof videos:   0%|          | 0/570 [00:00<?, ?it/s]

In [11]:
# Function to detect face and save bounding box
def detect_face(img_path):
    try:
        img = cv2.imread(str(img_path))
        if img is None:
            return False
            
        real_h, real_w = img.shape[:2]
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        boxes, probs = mtcnn.detect(img_rgb)
        
        if boxes is None or len(boxes) == 0:
            return False
            
        box = boxes[0]
        prob = probs[0]
        
        x1, y1, x2, y2 = box
        w = x2 - x1
        h = y2 - y1
        
        x = int(x1 * 224 / real_w)
        y = int(y1 * 224 / real_h)
        w = int(w * 224 / real_w)
        h = int(h * 224 / real_h)
        
        bb_path = img_path.parent / f"{img_path.stem}_BB.txt"
        with open(bb_path, 'w') as f:
            f.write(f"{x} {y} {w} {h} {prob:.7f}")
            
        return True
        
    except Exception as e:
        print(f"Error processing {img_path}: {str(e)}")
        return False

In [12]:
# Detect faces in all images
print("\nDetecting faces...")
for folder in ['live', 'spoof']:
    folder_path = dataset_root / folder
    images = list(folder_path.glob('*.jpg'))
    
    for img_path in tqdm(images, desc=f"Processing {folder} images"):
        if not detect_face(img_path):
            img_path.unlink()


Detecting faces...


Processing live images:   0%|          | 0/2947 [00:00<?, ?it/s]

Processing spoof images:   0%|          | 0/2840 [00:00<?, ?it/s]

In [13]:
# Function to rename files with new indices
def rename_files(folder_path, start_idx=1):
    files = sorted(f for f in folder_path.iterdir() if not f.name.endswith('_BB.txt'))
    
    # First rename to 7 digits
    print(f"Converting {folder_path.name} to 7 digits...")
    for idx, file in enumerate(tqdm(files)):
        ext = file.suffix
        # Rename image
        new_name = f"{(idx+1):07d}{ext}"
        new_path = file.parent / new_name
        file.rename(new_path)
        
        # Rename BB file if exists
        bb_file = file.parent / f"{file.stem}_BB.txt"
        if bb_file.exists():
            new_bb_name = f"{(idx+1):07d}_BB.txt"
            new_bb_path = file.parent / new_bb_name
            bb_file.rename(new_bb_path)
    
    # Then rename back to 6 digits
    files = sorted(f for f in folder_path.iterdir() if not f.name.endswith('_BB.txt'))
    print(f"Renaming {folder_path.name} to 6 digits...")
    for idx, file in enumerate(tqdm(files)):
        ext = file.suffix
        # Rename image
        new_name = f"{(start_idx+idx):06d}{ext}"
        new_path = file.parent / new_name
        file.rename(new_path)
        
        # Rename BB file if exists
        bb_file = file.parent / f"{file.stem}_BB.txt"
        if bb_file.exists():
            new_bb_name = f"{(start_idx+idx):06d}_BB.txt"
            new_bb_path = file.parent / new_bb_name
            bb_file.rename(new_bb_path)

In [14]:
# Rename all files
print("\nRenaming files...")
live_path = dataset_root / 'live'
spoof_path = dataset_root / 'spoof'

rename_files(live_path)  # Start live files from 000001
live_count = len(list(f for f in live_path.iterdir() if not f.name.endswith('_BB.txt')))
rename_files(spoof_path, start_idx=live_count+1)  # Continue numbering for spoof files


Renaming files...
Converting live to 7 digits...


  0%|          | 0/2659 [00:00<?, ?it/s]

Renaming live to 6 digits...


  0%|          | 0/2659 [00:00<?, ?it/s]

Converting spoof to 7 digits...


  0%|          | 0/2575 [00:00<?, ?it/s]

Renaming spoof to 6 digits...


  0%|          | 0/2575 [00:00<?, ?it/s]

In [15]:
# Create and upload new dataset
print("\nCreating Kaggle dataset...")
api = KaggleApi()
api.authenticate()

# Get username from Kaggle secrets
user_secrets = UserSecretsClient()
username = user_secrets.get_secret("kaggle-username")

metadata = {
    'title': 'Zalo-AIC - Face Anti-Spoofing Dataset',
    'id': f"{username}/zalo-aic-face-anti-spoofing-dataset",
    'licenses': [{'name': 'CC0-1.0'}]
}

metadata_path = dataset_root / 'dataset-metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=4)

print("Uploading dataset to Kaggle...")
api.dataset_create_new(
    folder=str(dataset_root),
    dir_mode='zip', 
    quiet=False
)

print("\nAll done!")


Creating Kaggle dataset...
Uploading dataset to Kaggle...
Starting upload for file spoof.zip


100%|██████████| 401M/401M [00:04<00:00, 85.5MB/s]


Upload successful: spoof.zip (401MB)
Starting upload for file live.zip


100%|██████████| 684M/684M [00:07<00:00, 90.3MB/s]


Upload successful: live.zip (684MB)

All done!
